In [ ]:
import pandas as pd # Import pandas for data manipulation and DataFrame operations
import glob # Import glob for finding files matching a pattern (e.g., all CSVs in a directory)
import os # Import os for operating system-related functionalities (e.g., path manipulation)

# --- Configuration ---
# Define the path to the directory containing the extracted CSV files from the scraper.
# It's assumed these CSVs are the raw output from the QS World Rankings scraper.
# IMPORTANT: Replace 'path to extracted csv files' with the actual relative or absolute path
path = '/Scraping_projects/QS_world_rankings'
print(f"Current working directory: {os.getcwd()}")

In [ ]:
# --- Configuration ---
# Define the path to the directory containing your raw extracted CSV files.
# This path is relative to where this script (process_rankings.py) is located.
# Assumed structure: .../QS_world_rankings/QS World Rankings/
CSV_INPUT_FOLDER = 'rankings_csv'

# Define the output folder for the combined and processed CSVs.
# This folder will be created within the same directory as this script.
COMBINED_OUTPUT_FOLDER = 'Combined_Rankings'

# Create the output folder if it doesn't exist
if not os.path.exists(COMBINED_OUTPUT_FOLDER):
    os.makedirs(COMBINED_OUTPUT_FOLDER)
    print(f"Created output directory for combined CSVs: {COMBINED_OUTPUT_FOLDER}")


# --- Define the mapping for Child Subject to Parent Subject ---
# This dictionary maps specific child subject strings (extracted from filenames)
# to their broader parent subject categories. This helps standardize the data.
# IMPORTANT: This map contains only a subset for demonstration. It is needed to
# populate this dictionary with ALL the relevant child subjects and their
# corresponding parent categories if you expand the scraping.
subject_to_parent_map = {
    "theology-divinity-religious-studies": "Arts & Humanities",
    #"veterinary-science": "Life Sciences & Medicine",
    #"computer-science": "Engineering & Technology",
    #"physics-astronomy": "Natural Sciences",
    "economics-econometrics": "Social Sciences & Management",
    "accounting-finance": "Social Sciences & Management",
    "business-management-studies": "Social Sciences & Management",
    "arts-and-humanities": "Arts & Humanities",
    "archaeology" : "Arts & Humanities", 
    #"life-sciences-medicine": "Life Sciences & Medicine",
    #"natural-sciences": "Natural Sciences",
    #"social-sciences-management": "Social Sciences & Management",
    # Add the rest of the child subjects and their parent categories here as needed:
    # "mathematics": "Natural Sciences",
     "history": "Arts & Humanities",
    # ... and so on for all subjects the scraper might extract.
}
print("Subject-to-Parent mapping defined (partial for demonstration).")

# --- File Discovery ---
# Use glob to find all files ending with '.csv' within the specified input folder.
# os.path.join ensures cross-platform compatibility when constructing paths.
all_files = glob.glob(os.path.join(CSV_INPUT_FOLDER, "*.csv"))
print(f"Found CSV files: {all_files}")

# Use a dictionary to store DataFrames, organized by year.
# This structure facilitates combining all subject-specific CSVs for a given year.
dfs_by_year = {}

# --- Process Each CSV File ---
# Iterate through each found CSV file to read, clean, and standardize its data.
for filepath in all_files:
    try:
        # Extract only the filename from the full path (e.g., 'QS_Rankings_subject-name-year.csv').
        filename = os.path.basename(filepath)
        print(f"\nProcessing file: {filename}")

        # Split the filename by underscores to parse out components.
        # Assumed filename format: 'QS_Rankings_SUBJECT-YEAR.csv'
        parts = filename.split('_')
        # Extract the subject and year from the last part of the filename.
        # .split('.')[0] removes the '.csv' extension.
        # .rsplit('-', 1) splits from the right, only once, to separate the subject name from the year.
        subject_and_year = parts[-1].split('.')[0]
        subject, year = subject_and_year.rsplit('-', 1)

        # Read the CSV file into a Pandas DataFrame.
        df = pd.read_csv(filepath)
        print(f"DataFrame loaded. Original columns: {df.columns.tolist()}")

        # --- Data Standardization and Feature Addition Logic ---
        # This section handles variations in the raw scraped data (e.g., missing 'Global Engagement' column)
        # and adds new categorical features ('Year', 'Parent Subject', 'Child Subject') to standardize the dataset.

        # 1. Ensure 'Global Engagement' column exists for consistent schema.
        if 'Global Engagement' not in df.columns:
            # If 'Global Engagement' is NOT present, add it as an empty column.
            # This accounts for variations in data provided by the website for certain subjects/years.
            df['Global Engagement'] = '' # Initialize with empty strings
            print("Added 'Global Engagement' column (was missing in this file).")

        # Dynamically assign 'Parent Subject' using the pre-defined mapping dictionary.
        # .get(key, default_value) provides a robust lookup, returning "Uncategorized" if a subject is not mapped.
        parent_subject_category = subject_to_parent_map.get(subject, "Uncategorized")
        print(f"Assigned Parent Subject: '{parent_subject_category}' for Child Subject: '{subject}'")

        # Determine the insertion index for new columns to maintain consistent order.
        # Defaults to -1 (end of DataFrame) if 'Global Engagement' column is not found (though it should be handled above).
        global_engagement_index = -1
        if 'Global Engagement' in df.columns:
            global_engagement_index = df.columns.get_loc('Global Engagement')

        # Insert 'Year', 'Parent Subject', and 'Child Subject' columns into the DataFrame.
        # These are inserted relative to the 'Global Engagement' column for standardized placement.
        df.insert(global_engagement_index + 1, 'Year', year)
        df.insert(global_engagement_index + 2, 'Parent Subject', parent_subject_category)
        df.insert(global_engagement_index + 3, 'Child Subject', subject)
        print("Inserted 'Year', 'Parent Subject', 'Child Subject' columns.")


        # --- Grouping DataFrames by Year ---
        # Store the processed DataFrame in the 'dfs_by_year' dictionary, grouped by its 'year'.
        if year not in dfs_by_year:
            dfs_by_year[year] = [] # Initialize a list for the year if it's encountered for the first time
        dfs_by_year[year].append(df) # Add the processed DataFrame to the list for its respective year

    except ValueError as e:
        # Catch ValueError, which might occur if filename parsing fails due to an unexpected format.
        print(f"Skipping file with unexpected name or format: {filepath}. Error: {e}")
        continue # Continue processing the next file in the list
    except KeyError as e:
        # Catch KeyError if a 'child subject' extracted from the filename is not found in the 'subject_to_parent_map'.
        print(f"Skipping file: '{filepath}'. Child subject '{subject}' not found in mapping. Error: {e}")
        continue # Continue processing the next file

# --- Combine and Save DataFrames by Year ---
# After processing all individual CSVs, this section combines them into a single DataFrame for each year.
# This creates a consolidated dataset per year, ready for further analysis or model training.
print("\nCombining and saving data by year...")
for year, dfs in dfs_by_year.items():
    # Concatenate all DataFrames for the current year into a single DataFrame.
    # ignore_index=True resets the index of the combined DataFrame, preventing duplicate indices.
    combined_df = pd.concat(dfs, ignore_index=True)

    # --- Optional: Reorder columns for final output consistency ---
    # This ensures a standardized column order across all combined CSVs, which is beneficial for
    # subsequent analysis and readability. Adjust 'desired_order' to your preferred final sequence.
    desired_order = [
        'Rank', 'Name', 'Location', 'Parent Subject', 'Child Subject', 'Year',
        'Employer Reputation', 'H-index Citations', 'Citations per Paper',
        'Academic Reputation', 'Global Engagement'
    ]
    # Filter and reorder columns, adding any new columns found in 'combined_df' that aren't in 'desired_order'
    # (these remaining columns will be appended at the end, sorted alphabetically).
    final_columns = [col for col in desired_order if col in combined_df.columns]
    remaining_columns = [col for col in combined_df.columns if col not in set(final_columns)]
    combined_df = combined_df[final_columns + sorted(remaining_columns)] # Reorder and append sorted remaining columns


    # Save the combined DataFrame for the year to a new CSV file.
    # The output file is placed in the 'COMBINED_OUTPUT_FOLDER' (e.g., './Combined_Rankings/').
    # index=False prevents pandas from writing the DataFrame index as a column in the CSV.
    output_filename = f'QS_Rankings_Combined_{year}.csv'
    output_filepath = os.path.join(COMBINED_OUTPUT_FOLDER, output_filename)
    combined_df.to_csv(output_filepath, index=False)
    print(f"Saved combined data for {year} to: {output_filepath}")

print("\nData wrangling complete. Combined CSVs are in the 'Combined_Rankings' folder.")